<a href="https://colab.research.google.com/github/rahiakela/hands-on-machine-learning-with-scikit-learn-keras-and-tensorflow/blob/18-reinforcement-learning/2_introduction_to_markov_decision_processes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Markov Decision Processes

In the early 20th century, **the mathematician Andrey Markov studied stochastic processes with no memory, called Markov chains. Such a process has a fixed number of states, and it randomly evolves from one state to another at each step. The probability for it to evolve from a state s to a state s′ is fixed, and it depends only on the pair (s, s′), not on past states (this is why we say that the system has no memory)**.

<img src='https://github.com/rahiakela/img-repo/blob/master/hands-on-machine-learning-keras-tensorflow/markov-chain.png?raw=1' width='800'/>

Suppose that the process starts in state $s_0$, and there is a 70% chance that it will remain in that state at the next step. Eventually it is bound to leave that state and never come back because no other state points back to $s_0$. If it goes to state $s_1$, it will then most likely go to state $s_2$ (90% probability), then immediately back to state $s_1$ (with 100% probability). It may alternate a number of times between these two states, but eventually it will fall into state $s_3$ and remain there forever (this is a terminal
state). Markov chains can have very different dynamics, and they are heavily used in thermodynamics, chemistry, statistics, and much more.



## Setup

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## Markov Chains

In [2]:
np.random.seed(42)

# shape=[s, s']
transition_probabilities = [
    [0.7, 0.2, 0.0, 0.1],  # from s0 to s0, s1, s2, s3
    [0.0, 0.0, 0.9, 0.1],  # from s1 to ...
    [0.0, 1.0, 0.0, 0.0],  # from s2 to ...
    [0.0, 0.0, 0.0, 1.0]   # from s3 to ...                       
]

n_max_steps = 50

def print_sequence():
  current_state = 0
  print('States:', end=' ')
  for step in range(n_max_steps):
    print(current_state, end=' ')
    if current_state == 3:
      break
    current_state = np.random.choice(range(4), p=transition_probabilities[current_state])
  else:
    print('...', end='')
  print()

for _ in range(10):
  print_sequence()

States: 0 0 3 
States: 0 1 2 1 2 1 2 1 2 1 3 
States: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 
States: 0 3 
States: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 
States: 0 1 3 
States: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 ...
States: 0 0 3 
States: 0 0 0 1 2 1 2 1 3 
States: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 


## Markov Decision Process

Markov decision processes were first [described in the 1950s by Richard Bellman](https://apps.dtic.mil/dtic/tr/fulltext/u2/606367.pdf).
**They resemble Markov chains but with a twist: at each step, an agent can choose one of several possible actions, and the transition probabilities depend on the chosen action.** Moreover, some state transitions return some reward (positive or negative), and the agent’s goal is to find a policy that will maximize reward over time.

For example, the MDP has three states (represented by circles) and up to three possible discrete actions at each step (represented by diamonds).

<img src='https://github.com/rahiakela/img-repo/blob/master/hands-on-machine-learning-keras-tensorflow/markov-decision-process.png?raw=1' width='800'/>

If it starts in state $s_0$, the agent can choose between actions $a_0, a_1$, or $a_2$. If it chooses action $a_1$, it just remains in state $s_0$ with certainty, and without any reward. It can thus decide to stay there forever if it wants to. But if it chooses action $a_0$, it has a 70% probability
of gaining a reward of +10 and remaining in state $s_0$. 

It can then try again and again to gain as much reward as possible, but at one point it is going to end up instead in state $s_1$. In state $s_1$ it has only two possible actions: $a_0$ or $a_2$. It can choose to stay put by repeatedly choosing action $a_0$, or it can choose to move on to state $s_2$ and get a negative reward of –50 (ouch). In state $s_2$ it has no other choice than to take action $a_1$, which will most likely lead it back to state $s_0$, gaining a reward of +40 on the way. 

You get the picture. By looking at this MDP, can you guess which strategy will
gain the most reward over time? In state $s_0$ it is clear that action $a_0$ is the best option, and in state $s_2$ the agent has no choice but to take action $a_1$, but in state $s_1$ it is not obvious whether the agent should stay put ($a_0$) or go through the fire ($a_2$).

Bellman found a way to estimate the optimal state value of any state $s$, noted $V*(s)$, which is the sum of all discounted future rewards the agent can expect on average after it reaches a state s, assuming it acts optimally.

He showed that if the agent acts optimally, then the Bellman Optimality Equation applies.This recursive equation says that if the agent acts optimally, then the optimal value of the current state is equal to the reward it will get on average after taking one optimal action, plus the expected optimal value of all possible next states that this action can lead to.

$$V^*(s) = max_a Σ_sT(s,a,s′)[R(s,a,s′) + γ · V*(s′)]$$

In this equation:

* $T(s, a, s′)$ is the transition probability from state $s$ to state $s′$, given that the agent chose action $a$. For example,$T(s_2, a_1, s_0) = 0.8$.

* $R(s, a, s′)$ is the reward that the agent gets when it goes from state $s$ to state $s′$, given that the agent chose action $a$. For example, $R(s2, a1,
s0) = +40.$

* $γ$ is the discount factor.



